In [1]:
import requests
import pandas as pd
import datetime
import os
from bs4 import BeautifulSoup as bs
from splinter import Browser

In [2]:
from webdriver_manager.chrome import ChromeDriverManager
exe_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **exe_path, headless=False)

[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Driver [/Users/mattdavies/.wdm/drivers/chromedriver/mac64/87.0.4280.88/chromedriver] found in cache
 


In [6]:
nasa_url = 'https://mars.nasa.gov/news/'
browser.visit(nasa_url)
html = browser.html
soup = bs(html, 'html.parser')

In [7]:
top = soup.find("div", class_="list_text")
news_title = top.find("div", class_= "content_title").text
news_p = top.find("div", class_="article_teaser_body").text

In [8]:
print(news_title,'\n', news_p)

NASA to Host Virtual Briefing on February Perseverance Mars Rover Landing 
 NASA leadership and members of the mission will discuss the agency’s latest rover, which touches down on the Red Planet on Feb. 18.


In [9]:
jpl_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(jpl_url)
html = browser.html
soup = bs(html, 'html.parser')

In [10]:
picture_url = soup.find('a', class_ = 'group cursor-pointer block')['href']

In [11]:
base_url = 'https://www.jpl.nasa.gov'
jpl_result = base_url + picture_url
jpl_result

'https://www.jpl.nasa.gov/images/southern-dunes-2/'

In [12]:
browser.visit(jpl_result)
html = browser.html
soup = bs(html, 'html.parser')

In [13]:
featured_image_url = soup.find('a', class_ = 'BaseButton text-contrast-none w-full mb-5 -primary -compact inline-block')['href']

In [14]:
featured_image_url

'https://d2pn8kiwq2w21t.cloudfront.net/original_images/jpegPIA24364.jpg'

In [15]:
url = "https://space-facts.com/mars/"
temp_df = pd.read_html(url)
mars_df = temp_df[0]
mars_df

,0,1
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [16]:
mars_df.columns=["Mars Profile", ""]
mars_df.set_index("Mars Profile", inplace=True)
mars_df.to_html("mars_facts.html")

In [17]:
usgs_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(usgs_url)
html = browser.html
soup = bs(html, 'html.parser')

In [18]:
link_list = []

for link in soup.find_all('a', class_ = 'itemLink product-item'):
    if link.get('href') not in link_list:
        link_list.append(link.get('href'))

link_list

['/search/map/Mars/Viking/cerberus_enhanced',
 '/search/map/Mars/Viking/schiaparelli_enhanced',
 '/search/map/Mars/Viking/syrtis_major_enhanced',
 '/search/map/Mars/Viking/valles_marineris_enhanced']

In [24]:
hemisphere_image_urls = []

In [25]:
for i in range(len(link_list)):
    browser.visit('https://astrogeology.usgs.gov' + link_list[i])
    html = browser.html
    soup = bs(html, 'html.parser')
    img_url = soup.find('a', text='Sample')['href']
    hemisphere_image_urls.append(
        {'hemisphere':link_list[i].split('/')[5].split('_')[0], 
        "img_url":img_url}
        )

In [26]:
hemisphere_image_urls

[{'hemisphere': 'cerberus',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
 {'hemisphere': 'schiaparelli',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'},
 {'hemisphere': 'syrtis',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'},
 {'hemisphere': 'valles',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]